In [41]:
import hashlib
import time

class Block:
    def __init__(self, index, transactions, previous_hash):
        self.index = index
        self.timestamp = time.time()
        self.transactions = transactions
        self.previous_hash = previous_hash
        self.hash = self.calculate_hash()

    def calculate_hash(self):
        return hashlib.sha256(f"{self.index}{self.timestamp}{self.previous_hash}".encode()).hexdigest()

class Transaction:
    def __init__(self, sender, recipient, amount, timestamp=None, input_utxos=None):
        self.sender = sender
        self.recipient = recipient
        self.amount = amount
        self.timestamp = timestamp or time.time()
        self.input_utxos = input_utxos or [] 
        self.tx_id = self.calculate_hash()

    def calculate_hash(self):
        return hashlib.sha256(f"{self.sender}{self.recipient}{self.amount}{self.timestamp}".encode()).hexdigest()

class Blockchain:
    def __init__(self):
        self.chain = []
        self.mempool = []
        self.utxo_set = {}  #{'amount': float, 'owner': str}
        self.create_genesis_block()

    def create_genesis_block(self):
        genesis_transaction = Transaction("network", "sana", 100)
        self.utxo_set[genesis_transaction.tx_id] = {'amount': 100, 'owner': 'sana'}
        self.mine_block("miner")

    def add_transaction_to_mempool(self, sender, recipient, amount):
        available_utxos = [(tx_id, utxo) for tx_id, utxo in self.utxo_set.items() if utxo['owner'] == sender]
        total_available = sum(utxo['amount'] for _, utxo in available_utxos)
        
        if total_available < amount:
            print(f"Transaction failed: {sender} does not have enough balance.")
            return False

        used_utxos = []
        used_amount = 0
        for tx_id, utxo in available_utxos:
            used_utxos.append(tx_id)
            used_amount += utxo['amount']
            if used_amount >= amount:
                break

        for utxo_id in used_utxos:
            del self.utxo_set[utxo_id]

        transaction = Transaction(sender, recipient, amount, time.time(), used_utxos)
        self.mempool.append(transaction)
        self.utxo_set[transaction.tx_id] = {'amount': amount, 'owner': recipient}
        
        change = used_amount - amount  #returning the change if btc have > btc spent/transferred
        if change > 0:
            change_tx_id = hashlib.sha256(f"{sender}{sender}{change}{time.time()}".encode()).hexdigest()
            self.utxo_set[change_tx_id] = {'amount': change, 'owner': sender}

        return True

    def mine_block(self, miner_address):
        if not self.mempool:
            print("No transactions to mine.")
            return

        valid_transactions = self.mempool[:]
        self.mempool = [] 

        # Mining reward transaction (set to zero)
        reward_tx = Transaction("network", miner_address, 0)
        valid_transactions.append(reward_tx)
        self.utxo_set[reward_tx.tx_id] = {'amount': 0, 'owner': miner_address}

        new_block = Block(len(self.chain), valid_transactions, self.chain[-1].hash if self.chain else "0")
        self.chain.append(new_block)

    def get_balance(self, owner):
        return sum(utxo['amount'] for utxo in self.utxo_set.values() if utxo['owner'] == owner)

def main():
    blockchain = Blockchain()
    
    print("\nInitial transactions:")
    blockchain.add_transaction_to_mempool("sana", "zain", 50)
    
    print("\nUpdated UTXO Set after transaction:")
    for tx_id, utxo in blockchain.utxo_set.items():
        print(f"{tx_id}: {utxo}")
    
    blockchain.mine_block("baryal")
    
    print("\nUpdated UTXO Set after mining:")
    for tx_id, utxo in blockchain.utxo_set.items():
        print(f"{tx_id}: {utxo}")

    print("\nInitial transactions:")
    blockchain.add_transaction_to_mempool("zain", "baryal", 25)
    
    print("\nUpdated UTXO Set after transaction:")
    for tx_id, utxo in blockchain.utxo_set.items():
        print(f"{tx_id}: {utxo}")
    
    print("\nFinal Balances:")
    print(f"Sana balance: {blockchain.get_balance('sana')}")
    print(f"Zain balance: {blockchain.get_balance('zain')}")
    print(f"Baryal balance: {blockchain.get_balance('baryal')}")

if __name__ == "__main__":
    main()

No transactions to mine.

Initial transactions:

Updated UTXO Set after transaction:
03cb8f7323e093057d6ad0160eb37bfa2b438edb4c01b59d22b799cdde135c61: {'amount': 50, 'owner': 'zain'}
c7ceb61d5fb760e18ef68dd707e7db424fbb65a35aa451d4d0aa6a4049f8c167: {'amount': 50, 'owner': 'sana'}

Updated UTXO Set after mining:
03cb8f7323e093057d6ad0160eb37bfa2b438edb4c01b59d22b799cdde135c61: {'amount': 50, 'owner': 'zain'}
c7ceb61d5fb760e18ef68dd707e7db424fbb65a35aa451d4d0aa6a4049f8c167: {'amount': 50, 'owner': 'sana'}
7e87fb059c43ddb99bbb33df07990f0864932aaca0cb9dad80eaaa3efeddbaab: {'amount': 0, 'owner': 'baryal'}

Initial transactions:

Updated UTXO Set after transaction:
c7ceb61d5fb760e18ef68dd707e7db424fbb65a35aa451d4d0aa6a4049f8c167: {'amount': 50, 'owner': 'sana'}
7e87fb059c43ddb99bbb33df07990f0864932aaca0cb9dad80eaaa3efeddbaab: {'amount': 0, 'owner': 'baryal'}
e2cf1095e28cf663de85c1c3fc7f367350033f929cb23ad7a9ef7ba10c05bab0: {'amount': 25, 'owner': 'baryal'}
6e75370024e546267d14d957a22380157b1